<a href="https://colab.research.google.com/github/bevanyeah/SIT796/blob/main/219296864_3_1D_Exact_policy_iteration_implementation_for_MDPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SIT796 Reinforcement Learning

Trimester one 2022

Task3.1D: Exact policy iteration implementation for MDPs

Bevan Fairleigh 219296864


In [1]:
!apt-get install -y xvfb x11-utils 
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyVirtualDisplay==2.0.* \
  PyOpenGL==3.1.* \
  PyOpenGL-accelerate==3.1.* \
  gym[box2d]==0.17.* 
!pip install pyglet


Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
import math, copy
import operator
import random
import pickle
import numpy as np
import gym
import base64
import io
import IPython
from gym.wrappers import Monitor
from IPython import display
from pyvirtualdisplay import Display

# Discretise states

In [3]:
pi = math.pi


#Function to give distribution of value for theta,
# that favour values closer to 0
def discrete_theta(min,max,theta_div):

  theta_state = []
  theta_state.append(float(round(max,2)))
  for _ in range(100):
    max /= theta_div
    if round(max,2) < 0.5:
      break
    theta_state.append(float(round(max,2)))
    theta_state.append(-float(round(max,2)))
  return theta_state


#Function to give distribution of value for velocity,
# that favour values closer to 0
def discrete_vel(min,max,vel_div):

  vel_states = []
  for _ in range(100):
    vel_states.append(float(round(max,2)))
    vel_states.append(-float(round(max,2)))
    max /= vel_div
    if round(max,0) < 1:
      break
  return vel_states


#Function to generate single dimention for all
# possible combination of states.
def generate_states(cos_1,cos_2,vel_1,vel_2):
  states = []
  for i in cos_1:
     for k in cos_2:
        for m in vel_1:
          for n in vel_2:
            states.append((i,k,m,n))
  return states

# Function to match raw observations to a valid 
# state values (discrete state values)
def raw2discrete(raw,states):
  states = np.asarray(states)
  match = states[(np.abs(states-raw)).argmin()]
  return match

#Function to match observations to a discrete state
# array 
def matchObs(converted_obs):
  matched_obs = (raw2discrete(converted_obs[0],theta_states),
                 raw2discrete(converted_obs[1],theta_states),
                 raw2discrete(converted_obs[2],vel1_states),
                 raw2discrete(converted_obs[3],vel2_states))
  return matched_obs

#Function to convert x and y coords to theta
# and build remaining RAW observations
def convert_obs(obs):
  converted_obs = [(float(round(math.atan2(obs[0],obs[1]),2))),(float(round(math.atan2(obs[2],obs[3]),2))),obs[4],obs[5]]

  return converted_obs


# Modifiers for determining theta and velocity
theta_div =1.7 
vel_div = 4

# Build all possible states for theta, and velocities

vel1_states = discrete_vel(-4*math.pi,4*math.pi,vel_div)
vel2_states = discrete_vel(-9*math.pi,9*math.pi,vel_div)
theta_states = [np.round(pi*(3/4),3),np.round(pi*(1/4),3),np.round(-pi*(3/4),3),np.round(-pi*(1/4),3)]

''' 
Reduced states alterative
Uncomment these to enable graphic view of MDP
'''

# vel1_states = [-12, 0, 12]
# vel2_states = [-28, 0, 28]
# theta_states = [np.round(pi/2,3),np.round(-pi/2,3)]

# Build states
states = generate_states(theta_states,theta_states,vel1_states,vel2_states)

# Build states as type string for our MDP
string_states = []
for state in states:
  string_states.append((' '.join(str(e) for e in state)))

 
actions = [0,1,2]

num_states= len(states)
num_actions = len(actions)

print(f"All theta states: {theta_states}")
print(f"All Velocity1 states: {vel1_states}")
print(f"All Velocity2 states: {vel2_states}")
print()
print(f"Generating a total of {len(states)} unique states")

All theta states: [2.356, 0.785, -2.356, -0.785]
All Velocity1 states: [12.57, -12.57, 3.14, -3.14, 0.79, -0.79]
All Velocity2 states: [28.27, -28.27, 7.07, -7.07, 1.77, -1.77]

Generating a total of 576 unique states


# Build Transition and Rewards Matrices & Dictionaries

In [4]:
#Build transistion and reward matrices/dictionaries


#Load our modified Acrobat Environment from local directory
# 
from acrobot import *
acrobot = AcrobotEnv()


#Function takes our states, actions and acrobot
# and loads each discrete state into bot.
# Upon performing each action, we observe the outcome
# and match the outcome to another discrete state.
# Output : Dictionary of Transistion function for MDP
#        : Matrix (num_states,num_states, actions) of transistion function
#        : Dictionary of rewards for MDP
#        : Matrix (num_states) of rewards

def trans_rewards(states,actions,acrobot):

  # Set our transition and rewards matrix and dictionaries
  trans_rewards = {}
  trans_probs = {}
  trans_matrix = np.zeros([num_states,num_states,num_actions])
  rewards_matrix = np.zeros(num_states)

  # For each state
  for i,state in enumerate(states):

    # Set environment to state
    # This is a method of our modified Acrobot
    acrobot.setState(state)
    state_string = (' '.join(str(e) for e in state))

    # If end state, ignore any transistions or rewards
    if acrobot._terminal():
      trans_rewards[state_string] = {}
      trans_probs[state_string] = {}
      continue

    #If we are here, we are not terminal, so set rewards matrix for current state as -1
    rewards_matrix[i]=-1

    #create dict for actions
    trans_probs[state_string] = {str(0):0,str(1):0,str(2):0}
    trans_rewards[state_string] = {str(0):0,str(1):0,str(2):0}

    # Attempt each action and observe results
    for j,action in enumerate(actions):

      action_string = str(action)

      #Take step with given action
      outcome = acrobot.step(action)

      #Break reward apart to examine
      reward = outcome[1]
      obs = outcome[0]

      # Convert our x,y back to theta, and build raw state
      converted_obs = convert_obs(obs)

      # Match our raw state to set discrete state
      matched_obs = matchObs(converted_obs)
      matched_obs_string = (' '.join(str(e) for e in matched_obs))

      #find in array for position of matched state
      position = states.index(matched_obs)

      #Set transistion probs for both dictionary and matrix
      action_dict = {}
      action_dict[action_string] = {matched_obs_string:1}
      trans_probs[state_string].update(action_dict)
      trans_matrix[i][position][j] = 1

      #set rewards for dictionary
      action_dict = {}
      action_dict[action_string] = {matched_obs_string:reward}
      trans_rewards[state_string].update(action_dict)
  
  return trans_probs,trans_rewards,trans_matrix,rewards_matrix

trans_probs_dict,trans_reward_dict,trans_matrix,rewards_matrix = trans_rewards(states,actions,acrobot)

# Build graphical MDP

In [5]:
'''
WARNING:  When using high state counts, this code takes a significant time to compute
'''

if num_states < 50:

  from mdp import MDP
  from mdp import has_graphviz
  import sys, os

  # Start at state closest to [0,0,0,0]
  startState = [0,0,0,0]
  matched_obs = matchObs(startState)
  matched_obs_string = (' '.join(str(e) for e in matched_obs))

  mdp = MDP(trans_probs_dict, trans_reward_dict, initial_state=matched_obs_string)
  
  if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
      !bash ../xvfb start
      os.environ['DISPLAY'] = ':1'

  import pydot
  from IPython.display import display
  print("Graphviz available:", has_graphviz)
  if has_graphviz:
      from mdp import plot_graph, plot_graph_with_state_values, plot_graph_optimal_strategy_and_state_values
      display(plot_graph(mdp))
else:
  print(f"Skipping MDP due to too many States : {num_states}")

Skipping MDP due to too many States : 576


# Policy Iteration

In [6]:
def policy_evaluation(policy, rewards_matrix, trans_matrix, gamma):

  # Create an empty Value array
  Value = np.zeros(num_states)

  # For each State
  for state in range(num_states):

    #Confirm current action from policy
    action = int(policy[state])

    #Resolve the current Value for the current State/Action pair 
    Value[state] = np.linalg.solve(np.identity(num_states) - gamma*trans_matrix[:,:,action], rewards_matrix)[state]

  return Value

def next_state_values(Value, trans_matrix, nextStateSelector):
  
  value_array = np.zeros(num_actions)
 
  #For each possible action, calculate the value, then return best action 
  for action in range(num_actions):

        # np.dot(nextStateSelector,trans) gives the nextstate of taking the action
        # then multiplied by Value, gives us the value of the next state
        # which gives us a (num_state x num_state), with only 1 value.  We take the sum to extract it from the matrix
        value_array[action] = np.sum(np.multiply(Value, np.dot(nextStateSelector, trans_matrix[:,:,action])))

  return value_array

def policyIteration(rewards_matrix,trans_matrix,policy):

  #Start with 0 value for all states
  Value = np.zeros(num_states)

  isChanged = True
  iterations = 0
  candidatePolicies = []
  candidateValues = []
  Value_sum = []
  countChanged = "Starting policy (0 changes)"

  while True:
    isChanged = False
    iterations += 1
    

    #Calculate current policy Value
    Value = policy_evaluation(policy, rewards_matrix, trans_matrix, gamma)

    # View current policy evaluation results
    print(f"Iteration: {iterations} | Policy value: {round(np.sum(Value),2)} | Policy changes: {countChanged}")


    '''
    With a number of state loops within our MDP, we found policy improvement could iterate
    repeatedly without converging.  So we have built a loop detector to exit when detected.

    '''

    # Add sum of values to list
    candidateValues.append(np.sum(Value))
    candidatePolicies.append(policy)

    #Loop detector triggers when identical sum of values is detected in list
    new_value = np.sum(Value)
    exist_count = Value_sum.count(new_value)

    #Exit policy iteration and return Policy
    if (exist_count==1):
      exist_pos = np.where(Value_sum == new_value)[0][0]
      print("Discovered loop in Policy improvement: Ending")
      candidateValues = candidateValues[exist_pos:]
      candidatePolicies = candidatePolicies[exist_pos:]
      break

    # If not found in list, append it for next iteration
    Value_sum.append(new_value)


    countChanged = 0
    for state in range(num_states):

        # A matrix function to enable selection of available next states
        nextStateSelector = np.zeros((1,num_states))
        nextStateSelector[0,state] = 1.0

        # Returns next state values for each action
        value_array = next_state_values(Value, trans_matrix, nextStateSelector)

        ''' 
        Instead of simple ArgMax, we use Argwhere to determine if there is any meaningful change in our policy
        Sometimes, a policy would change to another action which had the same value, which would help 
        create loops.  We now look at all values, and only change if another action provides a clear higher value.
        '''
        action = np.argwhere(value_array == np.amax(value_array))
        action = action.flatten().tolist()

        #  If there is more than one matching action
        if len(action)>1:

          # And the current policy is one of those, just continue without change
          if action.count(policy[state]) == 1:
            continue
          
          # Otherwise, update the policy with the first match
          else:
            policy[state] = action[0]
            isChanged = True
            countChanged+=1

        # Otherwise, if there is one clear action, make the change
        elif action[0] != policy[state]:
          policy[state] = action[0]
          isChanged = True
          countChanged+=1


    #If no change made to policy, we have converged without looping.
    if not isChanged:
      print("Policy is stable: Ending")
      break

  return(candidatePolicies[np.argmax(candidateValues)])

In [7]:

#Generate random policy
policy = np.random.randint(0, num_actions, size=num_states).astype(np.float32)

#Set Gamma high, as only end goal will be meaningful.
gamma = .5


policy = policyIteration(rewards_matrix,trans_matrix, policy)
  


Iteration: 1 | Policy value: -901.82 | Policy changes: Starting policy (0 changes)
Iteration: 2 | Policy value: -789.74 | Policy changes: 268
Iteration: 3 | Policy value: -781.1 | Policy changes: 96
Iteration: 4 | Policy value: -780.75 | Policy changes: 19
Iteration: 5 | Policy value: -780.76 | Policy changes: 3
Iteration: 6 | Policy value: -780.77 | Policy changes: 1
Policy is stable: Ending


In [8]:
np.save("policy", policy)

# Policy in real environment over 1000 episodes

In [9]:
policy = np.load("policy.npy")

env = gym.make('Acrobot-v1')
# env = Monitor(env,'./vid',force=True)

o = env.reset()

episodes = 1000
total_reward = 0

for _ in range(episodes):
    reward = 0
    while True:

      #Convert the observations to a known discrete state
      converted_obs = convert_obs(o)
      matched_obs = matchObs(converted_obs)

      #Find the arg value of the matched state to know the policy action
      position = states.index(matched_obs)

      action = int(policy[position])
      o, r, done, i = env.step(action)
      reward += r
      
      if done:
          total_reward += reward
          env.reset()
          break

print(f"Mean result over {episodes} episodes: {total_reward/episodes}")


Mean result over 1000 episodes: -267.88


# Policy in real environment export to video

In [10]:
policy = np.load("policy.npy")

!rm ./vid/*.*
!rmdir ./vid

d = Display()
d.start()

#Load the official Acrobot environment
env = gym.make('Acrobot-v1')
env = Monitor(env,'./vid',force=True)
count = 0
o = env.reset()

while True:

    #Convert the observations to a known discrete state
    converted_obs = convert_obs(o)
    matched_obs = matchObs(converted_obs)

    #Find the arg value of the matched state to know the policy action
    position = states.index(matched_obs)

    action = int(policy[position])
    o, r, done, i = env.step(action)


    if done:
        env.reset()
        count+=1
        if count == 10:
          break

env.close()
for f in env.videos:
    video = io.open(f[0], 'r+b').read()
    encoded = base64.b64encode(video)

    display.display(display.HTML(data="""
        <video alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        """.format(encoded.decode('ascii'))))

